In [2]:
import cv2
from matplotlib import category
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
# 1. 加载发票分类标准
def load_classification_standard(file_path):
    """
    加载发票分类表格，并将其转化为字典或数据框以供后续分类使用。
    """
    classification_df = pd.read_excel(file_path)
    # 修改列名
    classification_df.columns = ["category_id", "category", "items"]
    # 根据实际表格结构选择加载方式
    return classification_df


classification_file_path = "./whu/classification_standard.xlsx"
classification_standard = load_classification_standard(
        classification_file_path)


In [4]:
classification_standard.head()

,category_id,category,items
0,3020101,商品和服务支出_办公费_办公用品,计算器、纸类、本册、便签、财会用品、画具画材、签字笔、钢笔、笔芯、开关、电料、抽纸、卷纸、手...
1,3020102,商品和服务支出_其他商品和服务支出_书报杂志,各类报刊杂志
2,3020190,商品和服务支出_办公费_其他,洗洁精、油污清洁剂、除湿干燥剂、洁厕剂、家电清洁用品、消毒液、地板清洁剂、一次性清洁用品、拖...
3,3021302,商品和服务支出_维修(护)费_设备维修费,CPU、显卡、内存、主板、散热器、光驱、机箱、显示器、电源声卡、扩展卡、装机配件、SSD固态...
4,3021305,商品和服务支出_维修(护)费_网络运行与维护费,数据库、操作系统、防病毒、办公软件等


In [5]:
def build_classification_context(classification_df):
    """
    构建分类的上下文信息，将表格转换为大模型的输入上下文。
    """
    context = ""
    for index, row in classification_df.iterrows():
        # 假设表格中包含 "商品名称" 和 "分类" 列
        context += f"相关对应商品: {row['items']} -> 科目名称: {row['category']}\n"

    print(context)
    return context

# 构建分类上下文
classification_context = build_classification_context(classification_standard)



相关对应商品: 计算器、纸类、本册、便签、财会用品、画具画材、签字笔、钢笔、笔芯、开关、电料、抽纸、卷纸、手帕纸、湿巾、垃圾袋、垃圾桶、水壶、电话机、手电筒、电子钟表、电子灭蚊器、收音机、万用遥控器、对讲机、塑料杯、保温杯、杯具等 -> 科目名称: 商品和服务支出_办公费_办公用品
相关对应商品: 各类报刊杂志 -> 科目名称: 商品和服务支出_其他商品和服务支出_书报杂志
相关对应商品: 洗洁精、油污清洁剂、除湿干燥剂、洁厕剂、家电清洁用品、消毒液、地板清洁剂、一次性清洁用品、拖把、扫把、脸盆、水桶、抹布、百洁布、手套、鞋套、清洁刷具、其它清洁工具、清洁用具配件、饮用水等 -> 科目名称: 商品和服务支出_办公费_其他
相关对应商品: CPU、显卡、内存、主板、散热器、光驱、机箱、显示器、电源声卡、扩展卡、装机配件、SSD固态硬盘、移动硬盘、UPS电源、线缆、电脑工具、电脑清洁、插座、键盘、鼠标、摄像头、U盘、硬盘盒、手写板、鼠标垫、网络仪表仪器、投影配件、会议音频、硒鼓、墨粉、墨盒、墨水、色带、CD-R碟片、DVDR碟片、可擦写碟片、光盘包、光盘袋、光盘箱、光盘笔、光盘帖、迷你音响、LED屏配件、耳机、耳麦、麦克风、专业音频、苹果配件、无线音箱、存储卡、读卡器、相机包、电池、充电器、路由器、网络机柜、无线路由、无线网卡、网络存储、网络配件、防火墙等 -> 科目名称: 商品和服务支出_维修(护)费_设备维修费
相关对应商品: 数据库、操作系统、防病毒、办公软件等 -> 科目名称: 商品和服务支出_维修(护)费_网络运行与维护费
相关对应商品: 移液器、天平衡器、混合搅拌、清洗消毒、泵类仪器、干燥箱系列、培养箱系列、试验箱系列、振荡器、电炉系列、水浴水槽、分子生物学、萃取提纯、电泳成像、微生物学、粉碎合成、物理光学、光谱分析、色谱分析、PH计、酸度计、电导电阻、水质检测、多参数检测、环境监测、超纯水系统、净化安全、安全防护、急救清洗、石油仪器、食品检测、药物检测、煤炭纺织、特殊仪器、配件耗材、防噪音耳塞等 -> 科目名称: 商品和服务支出_专用材料费_实验室用品
相关对应商品: 羽毛球类、网球类、篮球类、乒乓类、足球类、桌球类、棒球类、铅球类、跳马、起跑器、跳高类、铁饼、标枪类、排球装备、其他体育用品 -> 科目名称: 商品和服务支出_专用材料费_消耗性体育用品
相

In [7]:
prompt = "根据发票上的商品名称，将其分类到相应的科目名称中。\n"
prompt += classification_context
print(prompt)

根据发票上的商品名称，将其分类到相应的科目名称中。
相关对应商品: 计算器、纸类、本册、便签、财会用品、画具画材、签字笔、钢笔、笔芯、开关、电料、抽纸、卷纸、手帕纸、湿巾、垃圾袋、垃圾桶、水壶、电话机、手电筒、电子钟表、电子灭蚊器、收音机、万用遥控器、对讲机、塑料杯、保温杯、杯具等 -> 科目名称: 商品和服务支出_办公费_办公用品
相关对应商品: 各类报刊杂志 -> 科目名称: 商品和服务支出_其他商品和服务支出_书报杂志
相关对应商品: 洗洁精、油污清洁剂、除湿干燥剂、洁厕剂、家电清洁用品、消毒液、地板清洁剂、一次性清洁用品、拖把、扫把、脸盆、水桶、抹布、百洁布、手套、鞋套、清洁刷具、其它清洁工具、清洁用具配件、饮用水等 -> 科目名称: 商品和服务支出_办公费_其他
相关对应商品: CPU、显卡、内存、主板、散热器、光驱、机箱、显示器、电源声卡、扩展卡、装机配件、SSD固态硬盘、移动硬盘、UPS电源、线缆、电脑工具、电脑清洁、插座、键盘、鼠标、摄像头、U盘、硬盘盒、手写板、鼠标垫、网络仪表仪器、投影配件、会议音频、硒鼓、墨粉、墨盒、墨水、色带、CD-R碟片、DVDR碟片、可擦写碟片、光盘包、光盘袋、光盘箱、光盘笔、光盘帖、迷你音响、LED屏配件、耳机、耳麦、麦克风、专业音频、苹果配件、无线音箱、存储卡、读卡器、相机包、电池、充电器、路由器、网络机柜、无线路由、无线网卡、网络存储、网络配件、防火墙等 -> 科目名称: 商品和服务支出_维修(护)费_设备维修费
相关对应商品: 数据库、操作系统、防病毒、办公软件等 -> 科目名称: 商品和服务支出_维修(护)费_网络运行与维护费
相关对应商品: 移液器、天平衡器、混合搅拌、清洗消毒、泵类仪器、干燥箱系列、培养箱系列、试验箱系列、振荡器、电炉系列、水浴水槽、分子生物学、萃取提纯、电泳成像、微生物学、粉碎合成、物理光学、光谱分析、色谱分析、PH计、酸度计、电导电阻、水质检测、多参数检测、环境监测、超纯水系统、净化安全、安全防护、急救清洗、石油仪器、食品检测、药物检测、煤炭纺织、特殊仪器、配件耗材、防噪音耳塞等 -> 科目名称: 商品和服务支出_专用材料费_实验室用品
相关对应商品: 羽毛球类、网球类、篮球类、乒乓类、足球类、桌球类、棒球类、铅球类、跳马、起跑器、跳高类、铁饼、标枪类、排球装备、其他体育用品 -> 科目名

In [ ]:
from zhipuai import ZhipuAI
api_key = ""  # 填写自己的APIKey
client = ZhipuAI(api_key=api_key) # 填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4-plus",  # 填写需要调用的模型编码
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "商品名称: 电脑"},
    ],
)
print(response.choices[0].message)

CompletionMessage(content='根据您提供的分类信息，"电脑"这一商品名称可以归类到以下两个科目名称中，具体取决于电脑的用途和采购目的：\n\n1. **商品和服务支出_维修(护)费_设备维修费**：\n   - 如果电脑是作为办公设备的一部分，用于日常办公或特定项目的设备维护和升级，可以归类到这个科目。\n\n2. **商品和服务支出_其他商品和服务支出_小额设备（科研项目）** 或 **商品和服务支出_其他商品和服务支出_小额设备（非科研项目）**：\n   - 如果电脑是作为新的设备采购，用于科研项目，则归类到“小额设备（科研项目）”。\n   - 如果电脑是用于非科研项目的日常办公或其他用途，则归类到“小额设备（非科研项目）”。\n\n具体归类需要根据实际使用情况和采购目的来确定。', role='assistant', tool_calls=None)


In [1]:
import json
import re

def extract_json_from_response(response_text):
    """
    从大模型的回答中提取出 JSON 格式的数据。

    参数:
    response_text (str): 大模型的回答文本。

    返回:
    dict: 提取出的 JSON 数据，解析为 Python 字典。
    """
    # 使用正则表达式提取 JSON 部分
    json_pattern = re.compile(r'\{.*?\}', re.DOTALL)
    match = json_pattern.search(response_text)
    
    if match:
        json_str = match.group(0)
        try:
            # 解析 JSON 字符串为 Python 字典
            json_data = json.loads(json_str)
            return json_data
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
            return None
    else:
        print("未找到 JSON 格式的数据")
        return None

# 示例使用
response_text = """
根据您提供的商品详细信息“*电子元件*八通道ADC芯片AD9257S”，我们可以将其分类到以下科目名称中：

**科目名称: 商品和服务支出_维修(护)费_设备维修费**

理由：
- **电子元件**：这类商品通常用于电子设备的维修或升级。
- **八通道ADC芯片AD9257S**：这是一种特定的电子元件，用于数据采集和转换，常见于各种电子设备和系统中。

因此，将其归类到“设备维修费”科目是合理的，因为这类元件的采购通常与设备的维修、升级或维护相关。

如果该元件是用于特定的科研项目或设备的研发，也可以考虑归类到“商品和服务支出_其他商品和服务支出_小额设备（科研项目）”科目，但基于提供的信息，归类到“设备维修费”更为直接和符合常规分类逻辑。

```json
{
  "商品详细信息": "*电子元件*八通道ADC芯片AD9257S",
  "科目名称": "商品和服务支出_维修(护)费_设备维修费",
  "理由": [
    "电子元件：这类商品通常用于电子设备的维修或升级。",
    "八通道ADC芯片AD9257S：这是一种特定的电子元件，用于数据采集和转换，常见于各种电子设备和系统中。",
    "因此，将其归类到“设备维修费”科目是合理的，因为这类元件的采购通常与设备的维修、升级或维护相关。",
    "如果该元件是用于特定的科研项目或设备的研发，也可以考虑归类到“商品和服务支出_其他商品和服务支出_小额设备（科研项目）”科目，但基于提供的信息，归类到“设备维修费”更为直接和符合常规分类逻辑。"
  ]
}
"""

In [2]:
json_pattern = re.compile(r'\{.*?\}', re.DOTALL)
match = json_pattern.search(response_text)

In [9]:
json_str = match.group(0)
json_data = json.loads(json_str)

In [10]:
json_data

{'商品详细信息': '*电子元件*八通道ADC芯片AD9257S',
 '科目名称': '商品和服务支出_维修(护)费_设备维修费',
 '理由': ['电子元件：这类商品通常用于电子设备的维修或升级。',
  '八通道ADC芯片AD9257S：这是一种特定的电子元件，用于数据采集和转换，常见于各种电子设备和系统中。',
  '因此，将其归类到“设备维修费”科目是合理的，因为这类元件的采购通常与设备的维修、升级或维护相关。',
  '如果该元件是用于特定的科研项目或设备的研发，也可以考虑归类到“商品和服务支出_其他商品和服务支出_小额设备（科研项目）”科目，但基于提供的信息，归类到“设备维修费”更为直接和符合常规分类逻辑。']}

In [16]:
from fileinput import filename
import shutil
import os

def handle_upload(file):
    file_name = os.path.basename(file)
    print(file_name)
    file_path = f"./tmp/temp_{file_name}"
    file = open(file, "rb")
    with open(file_path, "wb") as buffer:
        shutil.copyfileobj(file, buffer)
    return file_path

handle_upload("./whu/test/3.jpg")

3.jpg


'./tmp/temp_3.jpg'

In [17]:
file = open("/private/var/folders/tw/5jgn7f8j74zgq_93zn_8f2n40000gn/T/gradio/15f1dfdcf4c9ae7fd8a84953c3c376784a0caf7c2570c0d87cde09430d40300c/ZW_00_2024_PZ_B1ACD36F91274FBA98B293A5475AF429_1BODY_00003.jpg", "rb")

In [20]:
file = "/private/var/folders/tw/5jgn7f8j74zgq_93zn_8f2n40000gn/T/gradio/15f1dfdcf4c9ae7fd8a84953c3c376784a0caf7c2570c0d87cde09430d40300c/3.jpg"
file_path = "/Users/aliancn/code/python/Financial-identification/tmp/3.jpg"
shutil.copy(file, file_path)

'/Users/aliancn/code/python/Financial-identification/tmp/3.jpg'

In [7]:
# # coding=gb2312

import pandas as pd
import re
import os


# Extract relevant columns for creating regex patterns
def load_classification_data():
    file_path = os.path.join(os.getcwd(), './whu/classification_standard.xlsx') 
    excel_data = pd.ExcelFile(file_path)
    sheet_1 = excel_data.parse('商品对照')
    return sheet_1[['科目名称', '相关对应商品（达不到资产验收条件的商品，且不满足设备附件验收条件）']]


categories = load_classification_data()


# Example text to match against the regex patterns
example_text = "维修费"


def regex_match(categories, example_text):
    # Generate regex patterns based on the '相关对应商品' field for each category
    regex_patterns = {}
    for index, row in categories.iterrows():
        category = row['科目名称']
        items = row['相关对应商品（达不到资产验收条件的商品，且不满足设备附件验收条件）']
        # Split items by common delimiters (Chinese comma, comma, and other separators)
        item_keywords = re.split(r'[，、,]', str(items))
        # Create a regex pattern by joining keywords with '|'
        pattern = '|'.join([re.escape(keyword.strip())
                           for keyword in item_keywords if keyword.strip()])
        regex_patterns[category] = pattern

    # Check which category regex matches the example text
    matching_categories = [category for category, pattern in regex_patterns.items(
    ) if re.search(pattern, example_text)]

    # If no category matched, return a default message
    return matching_categories if matching_categories else ["未找到匹配类别"]


# Display the matching categories
print(regex_match(categories, example_text))


['未找到匹配类别']


使用easyocr获得画面信息，通过文字方式提供给大模型进行信息提取

In [ ]:
from pprint import pprint
import easyocr

reader = easyocr.Reader(['ch_sim', 'en'])

def extract_text_from_image(image_path):
    """
    从图像文件中提取文本信息。

    参数:
    image_path (str): 图像文件的路径。

    返回:
    str: 提取出的文本信息。
    """
    # 使用 EasyOCR 读取图像中的文本
    result = reader.readtext(image_path)
    content = ""
    for (bbox, text, prob) in result:
        bbox_str = ",".join([str(coord) for coord in bbox])
        content += f"({bbox_str} {text})\n"
    # convert to string
    return content


content = extract_text_from_image("./whu/test/3.jpg")
pprint(content)


('([767, 160],[1502, 160],[1502, 257],[767, 257] 电子发瓢(晋通发票))\n'
 '([1703, 197],[2223, 197],[2223, 241],[1703, 241] 发票号码:  '
 '24422000000113017597)\n'
 '([1044, 238],[1076, 238],[1076, 270],[1044, 270] s)\n'
 '([1701, 262],[2115, 262],[2115, 305],[1701, 305] 开票日期: 2024年08月30日)\n'
 '([1080, 314],[1256, 314],[1256, 364],[1080, 364] 湖北省税务局)\n'
 '([148, 439],[386, 439],[386, 483],[148, 483] 名称:武汊大学)\n'
 '([1234, 441],[1676, 441],[1676, 485],[1234, 485] 名称:湖北综研技术有限公司)\n'
 '([95, 431],[131, 431],[131, 623],[95, 623] ")\n'
 '([146, 550],[1114, 550],[1114, 598],[146, 598] '
 '|统一社会信用代码/纳税人识别号:12100000707137123卫)\n'
 '([1179, 433],[1221, 433],[1221, 587],[1179, 587] #)\n'
 '([1233, 551],[2206, 551],[2206, 601],[1233, 601] '
 '|统一社会信用代码/纳税人识别号:91420100MA4 SMFTP2卫)\n'
 '([1182, 584],[1212, 584],[1212, 622],[1182, 622] 息)\n'
 '([201, 651],[345, 651],[345, 693],[201, 693] 项目名称)\n'
 '([483, 651],[627, 651],[627, 693],[483, 693] 规格型号)\n'
 '([753, 649],[859, 649],[859, 691],[753, 691] 单  位)\n'
 '([103

In [19]:
prompt = """
#### 背景信息：
您需要从发票图像的文本识别结果中提取以下关键信息，并将其填充到指定的字段中。以下是目标字段和说明：
- **InvoiceNum**: 发票号，通常以 "发票号码" 为标识，后接一串数字。
- **InvoiceDate**: 开票日期，通常以 "开票日期" 或类似标识开头，后接日期（格式如 YYYY年MM月DD日 或 YYYY-MM-DD）。
- **NoteDrawer**: 开票人名称，可能出现在与发票签名或备注相关的字段中。
- **TotalAmount**: 发票总金额，可能标识为 "金额"、"价税合计" 或类似关键字。
- **TotalTax**: 发票总税额，通常与 "税额" 或 "合计税额" 相关。
- **SellerName**: 销售方名称，可能标识为 "销售方" 或 "名称"。
- **SellerRegisterNum**: 销售方纳税人识别号，可能标识为 "纳税人识别号"。
- **PurchaserName**: 购买方名称，可能标识为 "购买方" 或 "名称"。
- **PurchaserRegisterNum**: 购买方纳税人识别号，可能标识为 "纳税人识别号"。
- **CommodityDetails**: 包括发票中商品详情，通常以表格形式列出，包括商品名称、数量、单价、金额等。
您提供的输入是由 OCR 输出的文字和坐标列表，如下：
```
[([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], "文本内容")]
```
#### 任务目标：
根据输入的列表，推断每个字段在发票中的位置，并提取所需信息，输出如下结构化字典：
```python
fields = {
    "InvoiceNum": "24422000000113017597",
    "InvoiceDate": "2024年08月30日",
    "NoteDrawer": "开票人姓名",
    "TotalAmount": "1000.00",
    "TotalTax": "100.00",
    "SellerName": "湖北综研技术有限公司",
    "SellerRegisterNum": "123456789012345",
    "PurchaserName": "武汊大学",
    "PurchaserRegisterNum": "987654321098765",
    "CommodityDetails": [
        {"Name": "商品1", "Quantity": "2", "UnitPrice": "50.00", "Amount": "100.00"},
        {"Name": "商品2", "Quantity": "1", "UnitPrice": "900.00", "Amount": "900.00"}
    ]
}
```
#### 提取逻辑：
1. **关键字段匹配**：根据以下关键字匹配发票字段内容：
   - 发票号：以 "发票号码" 开头，提取后续数字部分。
   - 开票日期：以 "开票日期" 开头，提取日期内容。
   - 销售方名称：以 "销售方" 或 "名称" 为标识，提取后续名称。
   - 销售方纳税人识别号：以 "纳税人识别号" 为标识，提取后续数字。
   - 购买方名称：以 "购买方" 或 "名称" 为标识，提取后续名称。
   - 购买方纳税人识别号：以 "纳税人识别号" 为标识，提取后续数字。
   - 总金额和总税额：查找关键字 "金额" 或 "价税合计" 和 "税额"，提取后续数字。
2. **坐标区域辅助判断**：通过坐标位置推断字段的空间分布关系，确保提取内容的正确性。
   - 通常同一字段的子信息会位于相邻区域。
   - 表格内容（如商品详情）通常位于统一表格区域，可根据纵向或横向坐标推断。
3. **表格处理（商品详情）**：识别商品详情表格区域，根据列名提取商品名称、数量、单价、金额等信息，并填充 `CommodityDetails` 字段。
#### 特别注意：
- 若部分字段无法从输入中提取，请标记为 `""`（空字符串）。
- 若表格字段中存在多个商品条目，需将其逐行添加至 `CommodityDetails`。
"""

In [20]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
from zhipuai import ZhipuAI
api_key = os.environ.get("ZHIPU_API_KEY")
client = ZhipuAI(api_key=api_key) # 填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4-plus",  # 填写需要调用的模型编码
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": content},
    ],
)
print(response.choices[0].message)

CompletionMessage(content='根据提供的 OCR 输出文本和坐标信息，我们需要提取发票的关键信息并填充到指定的字段中。以下是提取过程和结果：\n\n### 提取过程\n\n1. **发票号 (InvoiceNum)**:\n   - 文本: "发票号码:  24422000000113017597"\n   - 提取: "24422000000113017597"\n\n2. **开票日期 (InvoiceDate)**:\n   - 文本: "开票日期: 2024年08月30日"\n   - 提取: "2024年08月30日"\n\n3. **开票人名称 (NoteDrawer)**:\n   - 文本: "开票人:  魏友琼"\n   - 提取: "魏友琼"\n\n4. **总金额 (TotalAmount)**:\n   - 文本: "(小写)垩9850.00"\n   - 提取: "9850.00"\n\n5. **总税额 (TotalTax)**:\n   - 文本: "垩97.52"\n   - 提取: "97.52"\n\n6. **销售方名称 (SellerName)**:\n   - 文本: "名称:湖北综研技术有限公司"\n   - 提取: "湖北综研技术有限公司"\n\n7. **销售方纳税人识别号 (SellerRegisterNum)**:\n   - 文本: "|统一社会信用代码/纳税人识别号:91420100MA4 SMFTP2卫"\n   - 提取: "91420100MA4SMFTP2"\n\n8. **购买方名称 (PurchaserName)**:\n   - 文本: "名称:武汊大学"\n   - 提取: "武汊大学"\n\n9. **购买方纳税人识别号 (PurchaserRegisterNum)**:\n   - 文本: "|统一社会信用代码/纳税人识别号:12100000707137123卫"\n   - 提取: "12100000707137123"\n\n10. **商品详情 (CommodityDetails)**:\n    - 根据表格列名和对应数据提取：\n      - 商品1:\n        - 名称: "k电子元件*八通道ADC芯"\n        - 数量: "10"\n

In [22]:
pprint(response.choices[0].message.content)

('根据提供的 OCR 输出文本和坐标信息，我们需要提取发票的关键信息并填充到指定的字段中。以下是提取过程和结果：\n'
 '\n'
 '### 提取过程\n'
 '\n'
 '1. **发票号 (InvoiceNum)**:\n'
 '   - 文本: "发票号码:  24422000000113017597"\n'
 '   - 提取: "24422000000113017597"\n'
 '\n'
 '2. **开票日期 (InvoiceDate)**:\n'
 '   - 文本: "开票日期: 2024年08月30日"\n'
 '   - 提取: "2024年08月30日"\n'
 '\n'
 '3. **开票人名称 (NoteDrawer)**:\n'
 '   - 文本: "开票人:  魏友琼"\n'
 '   - 提取: "魏友琼"\n'
 '\n'
 '4. **总金额 (TotalAmount)**:\n'
 '   - 文本: "(小写)垩9850.00"\n'
 '   - 提取: "9850.00"\n'
 '\n'
 '5. **总税额 (TotalTax)**:\n'
 '   - 文本: "垩97.52"\n'
 '   - 提取: "97.52"\n'
 '\n'
 '6. **销售方名称 (SellerName)**:\n'
 '   - 文本: "名称:湖北综研技术有限公司"\n'
 '   - 提取: "湖北综研技术有限公司"\n'
 '\n'
 '7. **销售方纳税人识别号 (SellerRegisterNum)**:\n'
 '   - 文本: "|统一社会信用代码/纳税人识别号:91420100MA4 SMFTP2卫"\n'
 '   - 提取: "91420100MA4SMFTP2"\n'
 '\n'
 '8. **购买方名称 (PurchaserName)**:\n'
 '   - 文本: "名称:武汊大学"\n'
 '   - 提取: "武汊大学"\n'
 '\n'
 '9. **购买方纳税人识别号 (PurchaserRegisterNum)**:\n'
 '   - 文本: "|统一社会信用代码/纳税人识别号:12100000707137123卫"\n'
 '   - 提取: "1210000070713712

In [1]:
prompt_1233 = """
**输入**：  
1. 发票信息：包含商品/服务名称、金额、日期、发票种类等。  
2. 用户信息：如职务、部门、报销账户余额、历史报销记录等。  
3. 报销政策：包含政策分类、适用范围、报销比例、金额上限、有效期等规则。

**任务**：  
- 判断发票内容是否符合报销政策要求。  
- 计算实际可报销金额，并指出超限额或特殊审批需求。  
- 提供政策适用的具体条款依据及注意事项。  

**输出**：  
- 报销判断结果（可报销/不可报销）及理由。  
- 适用的政策条款及依据。  
- 可报销金额及相关限制说明。  
- 必要的操作建议或提交材料清单（如需额外审批）。  

**要求**：  
确保推理结果准确且易于理解，反馈信息结构化（JSON 格式），支持多样化政策和复杂报销场景。
"""


def get_prompt_for_as_rag(invoice_info, uer_info, policy_info):
    """
    根据输入的发票信息、用户信息和报销政策信息，生成对应的任务描述。

    参数:
    invoice_info (str): 发票信息。
    uer_info (str): 用户信息。
    policy_info (str): 报销政策信息。

    返回:
    str: 生成的任务描述。
    """
    prompt = f"""
    **输入**：  
    1. 发票信息：
    {invoice_info}  
    2. 用户信息：
    {uer_info}
    3. 报销政策：
    {policy_info}

    **任务**：  
    - 判断发票内容是否符合报销政策要求。  
    - 计算实际可报销金额，并指出超限额或特殊审批需求。  
    - 提供政策适用的具体条款依据及注意事项。  

    **输出**：  
    - 报销判断结果（可报销/不可报销）及理由。  
    - 适用的政策条款及依据。  
    - 可报销金额及相关限制说明。  
    - 必要的操作建议或提交材料清单（如需额外审批）。  

    **要求**：  
    确保推理结果准确且易于理解，反馈信息结构化（JSON 格式），支持多样化政策和复杂报销场景。
    """
    
    return prompt


invoice_info_example = """
{
    "InvoiceNum": "24952000000110579895",
    "InvoiceDate": "2024年07月17日",
    "NoteDrawer": "吴启山",
    "TotalAmount": "121.81",
    "TotalTax": "15.83",
    "SellerName": "深圳市浩凌科技有限公司",
    "SellerRegisterNum": "91440300587908164C",
    "PurchaserName": "武汉大学",
    "PurchaserRegisterNum": "12100000707137123P",
    "CommodityDetails": [
        "*计算机配套产品*键盘"
    ],
    "totalPay": "137.64"
}
"""

user_info_example = """
{
    "UserName": "张三",
    "Department": "信息管理中心",
    "post" : "副教授",
    "Balance": "1000.00",
}
"""

from rag_policy.byhand import getTextPdf

policy_info_example = getTextPdf("./rag_policy/policy.pdf")
from pprint import pprint
pr = get_prompt_for_as_rag(invoice_info_example, user_info_example, policy_info_example)
pprint(pr)

    


('\n'
 '    **输入**：  \n'
 '    1. 发票信息：\n'
 '    \n'
 '{\n'
 '    "InvoiceNum": "24952000000110579895",\n'
 '    "InvoiceDate": "2024年07月17日",\n'
 '    "NoteDrawer": "吴启山",\n'
 '    "TotalAmount": "121.81",\n'
 '    "TotalTax": "15.83",\n'
 '    "SellerName": "深圳市浩凌科技有限公司",\n'
 '    "SellerRegisterNum": "91440300587908164C",\n'
 '    "PurchaserName": "武汉大学",\n'
 '    "PurchaserRegisterNum": "12100000707137123P",\n'
 '    "CommodityDetails": [\n'
 '        "*计算机配套产品*键盘"\n'
 '    ],\n'
 '    "totalPay": "137.64"\n'
 '}\n'
 '  \n'
 '    2. 用户信息：\n'
 '    \n'
 '{\n'
 '    "UserName": "张三",\n'
 '    "Department": "信息管理中心",\n'
 '    "post" : "副教授",\n'
 '    "Balance": "1000.00",\n'
 '}\n'
 '\n'
 '    3. 报销政策：\n'
 '    设备报销政策（编）\n'
 '⼤学仪器设备报销管理⽅案\n'
 '第⼀章 总则\n'
 '第⼀条 ⽬的与适⽤范围\n'
 '为规范⼤学仪器设备的采购及报销流程，提⾼资⾦使⽤效益，特制定本⽅案。适⽤于\n'
 '全校范围内教职⼯及学⽣因教学、科研、实验等⼯作需要购买仪器设备的报销管理。\n'
 '第⼆条 原则\n'
 '\ue072\ue094\x00合规性原则：采购和报销需符合学校财务制度及相关法律法规。\n'
 '\ue073\ue094\x00合理性原则：采购的设备应与实际教学、科研需求相符。\n'
 '\ue074\ue094\x00透明性原则